In [1]:
import config
import sqlalchemy
import functions
import statsapi as mlb

In [2]:
gamePks = functions.read_gamePks()

In [12]:
test_pk = gamePks['2019'][101]

## SQL Alchemy engine

In [4]:
_sql_alchemy_connection = (
                                f'mysql+mysqlconnector://'
                                f'{config.user}:{config.password}'
                                f'@{config.host}:{config.port}'
                                f'/{config.schema}'
                           )

db = sqlalchemy.create_engine(_sql_alchemy_connection,
                              echo = False,
                              connect_args = {'ssl_disabled' : True})

In [5]:
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

from sqlalchemy import Column, Integer, String,DateTime,Date,Time,Boolean
# Declare mapping for the game table 
class Game(Base):
    __tablename__ = 'games'
    __table_args__ = {'extend_existing': True}
    
    pk = Column(Integer)
    type = Column(String(1))
    doubleHeader = Column(String(1))
    id = Column(String(150), primary_key=True,unique=True)
    gamedayType = Column(String(1))
    tiebreaker = Column(String(1))
    gameNumber = Column(Integer)
    calenderEventId = Column(String(50))
    season = Column(Integer)
    
    dateTime = Column(DateTime)
    originalDate = Column(Date)
    dayNight = Column(String(12))
    time = Column(Time)
    
    abstractGameState = Column(String(12))
    codedGameState = Column(String(3))
    detailedState = Column(String(12))
    statusCode = Column(String(3))
    abstractGameCode = Column(String(3))
    
    homeTeam_id = Column(Integer)
    awayTeam_id = Column(Integer)
    
    condition = Column(String(25))
    temp = Column(Integer)
    wind = Column(String(50))
    
    venue_id = Column(Integer)
    
    home_probablePitcher = Column(Integer)
    away_probablePitcher = Column(Integer)
    
    def __repr__(self): 
        return "<Game(pk='%s',id='%s')>" % (
                        self.pk, self.id)

In [6]:
db.table_names()

['games', 'plays']

In [7]:
# Create the table 
Base.metadata.create_all(db)

In [7]:
# Create instance of the mapped class 
def create_gameRecord(pk):
    from datetime import datetime
    import time
    api_call = mlb.get('game',{'gamePk':pk})
    gameData = api_call['gameData']
    game = gameData['game']
    _datetime = gameData['datetime']
    status = gameData['status']
    weather = gameData['weather']
    probablePitchers = gameData['probablePitchers']
    
    game_record = Game(pk=game['pk'],
                 type=game['type'],
                 doubleHeader=game['doubleHeader'],
                 id=game['id'],
                 gamedayType=game['gamedayType'],
                 tiebreaker=game['tiebreaker'],
                 gameNumber=game['gameNumber'],
                 calenderEventId=game['calendarEventID'],
                 season=game['season'],
                  
                 dateTime=datetime.strptime(_datetime['dateTime'],'%Y-%m-%dT%H:%M:%SZ'),
                 originalDate=datetime.strptime(_datetime['originalDate'],"%Y-%m-%d"),
                 dayNight=_datetime['dayNight'],
                 time=datetime.strptime(_datetime['time']+_datetime['ampm'],"%H:%M%p"),
                  
                 abstractGameState=status['abstractGameState'],
                 codedGameState=status['codedGameState'],
                 detailedState=status['detailedState'],
                 statusCode=status['statusCode'],
                 abstractGameCode=status['abstractGameCode'],
                  
                 homeTeam_id=gameData['teams']['home']['id'],
                 awayTeam_id=gameData['teams']['away']['id'],
                  
                 condition=weather.get('condition','null'),
                 temp=weather.get('temp','null'),
                 wind=weather.get('wind','null'),
                  
                 venue_id=gameData['venue']['id'],
                  
                 home_probablePitcher=probablePitchers.get('home',{'null':'null'}).get('id','null'),
                 away_probablePitcher=probablePitchers.get('away',{'null':'null'}).get('id','null') 
                 )
    return game_record

In [21]:
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

class Play(Base):
    __tablename__= 'plays'
    __table_args__ = {'extend_existing': True} 
    
    id = Column(String(200),primary_key=True,unique=True)
    type = Column(String(10))
    event = Column(String(25))
    eventType = Column(String(25))
    description = Column(String(250))
    rbi = Column(Integer)
    awayScore = Column(Integer)
    homeScore = Column(Integer)
    
    atBatIndex = Column(Integer)
    halfInning = Column(String(10))
    inning = Column(Integer)
    startTime = Column(DateTime)
    endTime = Column(DateTime)
    isComplete = Column(Boolean)
    isScoringPlay = Column(Boolean)
    hasReview = Column(Boolean)
    hasOut = Column(Boolean)
    captivatingIndex = Column(Integer)
    
    batter_id = Column(Integer)
    pitcher_id = Column(Integer)
    
    count_balls = Column(Integer)
    count_strikes = Column(Integer)
    count_outs = Column(Integer)
    
    num_pitches = Column(Integer)
    num_actions = Column(Integer)
    num_runners = Column(Integer)
    
    #game_pk = Column(Integer,ForeignKey('games.pk'),nullable=False)
    game_id = Column(String(150),ForeignKey('games.id'))
    
    game = relationship("Game",back_populates="plays")
    
    def __repr__(self):
        return "<Play(game_id='%s',atBatIndex='%s')>" % (
                     self.game_id,self.atBatIndex)

Game.plays = relationship(
    "Play",order_by=Play.id,back_populates='game')

In [17]:
Base.metadata.create_all(db)

In [9]:
# Create instance of the mapped class 
def create_gameRecord_playsRecords(pk,commit=True):
    from datetime import datetime
    import time
    already_added_pk = [item for sublist in db.execute('select pk from games').fetchall() for item in sublist]
    
    if int(pk) not in already_added_pk:
        api_call = mlb.get('game',{'gamePk':pk})

        gameData = api_call['gameData']
        game = gameData['game']
        _datetime = gameData['datetime']
        status = gameData['status']
        weather = gameData['weather']
        probablePitchers = gameData['probablePitchers']

        liveData = api_call['liveData']
        all_plays = liveData['plays']['allPlays']
    
        already_added = [item for sublist in db.execute('select id from games').fetchall() for item in sublist]
    

        game_record = Game(pk=game['pk'],
                     type=game['type'],
                     doubleHeader=game['doubleHeader'],
                     id=game['id'],
                     gamedayType=game['gamedayType'],
                     tiebreaker=game['tiebreaker'],
                     gameNumber=game['gameNumber'],
                     calenderEventId=game['calendarEventID'],
                     season=game['season'],

                     dateTime=datetime.strptime(_datetime['dateTime'],'%Y-%m-%dT%H:%M:%SZ'),
                     originalDate=datetime.strptime(_datetime['originalDate'],"%Y-%m-%d"),
                     dayNight=_datetime['dayNight'],
                     time=datetime.strptime(_datetime['time']+_datetime['ampm'],"%H:%M%p"),

                     abstractGameState=status['abstractGameState'],
                     codedGameState=status['codedGameState'],
                     detailedState=status['detailedState'],
                     statusCode=status['statusCode'],
                     abstractGameCode=status['abstractGameCode'],

                     homeTeam_id=gameData['teams']['home']['id'],
                     awayTeam_id=gameData['teams']['away']['id'],

                     condition=weather.get('condition','null'),
                     temp=weather.get('temp','null'),
                     wind=weather.get('wind','null'),

                     venue_id=gameData['venue']['id'],

                     home_probablePitcher=probablePitchers.get('home',{'null':'null'}).get('id','null'),
                     away_probablePitcher=probablePitchers.get('away',{'null':'null'}).get('id','null') 
                     )
        play_records = []
        for play in all_plays:
            result = play['result']
            about = play['about']
            batter = play.get('matchup',{'batter':'null'})['batter']
            pitcher = play.get('matchup',{'pitcher':'null'})['pitcher']
            count = play['count']

            play_record = Play(id=str(game['pk'])+game['id']+str(about['atBatIndex']),
                              type=result['type'],
                              event=result['event'],
                              eventType=result.get('eventType','null'),
                              description=result.get('description','null'),
                              rbi=result.get('rbi','null'),
                              awayScore=result.get('awayScore','null'),
                              homeScore=result.get('homeScore','null'),

                              atBatIndex=about.get('atBatIndex','null'),
                              halfInning=about['halfInning'],
                              inning=about['inning'],
                              startTime=datetime.strptime(about.get('startTime','1900-01-01T01:01:1.0Z'),'%Y-%m-%dT%H:%M:%S.%fZ'),
                              endTime=datetime.strptime(about.get('endTime','1900-01-01T01:01:01.0Z'),'%Y-%m-%dT%H:%M:%S.%fZ'),
                              isComplete=about.get('isComplete','null'),
                              isScoringPlay=about.get('isScoringPlay','null'),
                              hasReview=about.get('hasReview','null'),
                              hasOut=about.get('hasOut','null'),
                              captivatingIndex=about.get('captivatingIndex','null'),

                              batter_id=batter.get('id','null'),
                              pitcher_id=pitcher.get('id','null'),

                              count_balls=count.get('balls','null'),
                              count_strikes=count.get('striks','null'),
                              count_outs=count.get('outs','null'),

                              num_pitches=len(play['pitchIndex']),
                              num_actions=len(play['actionIndex']),
                              num_runners=len(play['runners']),

                              game_id=game['id'] 
                              )
            play_records.append(play_record)
            
        if commit:
            session.add(game_record)
            session.commit()

            session.add_all(play_records)
            session.commit()
    else:
        pass
    #return game_record,play_records

In [8]:
# Create a Session 
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=db)
session = Session()


In [17]:
## insert games and plays
# [create_gameRecord_playsRecords(x) for x in gamePks['2019']]

In [ ]:
class Team(Base):
    

In [ ]:
class Roster(Base):
    __tablename__ = 'rosters'
    __table_args__ = {'extend_existing': True}
    
    date = Column(Date)
    team_id = Column(Integer,ForeignKey)
    player_1 =
    player_2
    player_3
    player_4
    player_5
    player_6
    player_7
    player_8
    player_9
    player_10
    player_11
    player_12
    player_13
    player_14
    player_15
    player_16
    player_17
    player_18
    player_19
    player_20
    player_21
    player_22
    player_23
    player_24
    player_25
    player_26
    player_27
    player_28
    player_29
    player_30
    player_31
    player_32
    player_33
    player_34
    player_35
    player_36
    player_37
    player_38
    player_39
    player_40
    
    game_id = Column(String(150),ForeignKey('games.id'))
    